In [ ]:
import math
import random
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display

TITLE = '‹bung: Elektrischer Leiter'

MATERIALS = [
    {'name': 'Kupfer', 'rho': 0.0178, 'alpha': 0.0039},      # Ohm∑mm≤/m
    {'name': 'Aluminium', 'rho': 0.0282, 'alpha': 0.0040},   # Ohm∑mm≤/m
]

L_VALUES = [0.01, 0.02, 0.05,
            0.1, 0.2, 0.5,
            1.0, 2.0, 5.0,
            10.0, 20.0, 50.0]

V_VALUES = [1.0, 5.0, 12.0, 24.0, 230.0, 400.0]
I_VALUES = [1.0, 2.0, 5.0, 10.0, 16.0, 20.0, 32.0, 50.0]
RL_VALUES = [0.5, 1.0, 2.0, 5.0, 10.0, 20.0, 50.0, 100.0]
GEOM_VALUES_MM = [0.5, 1.0, 2.0, 5.0, 10.0, 20.0]
S_TARGET_VALUES = [0.1, 0.2, 0.5, 1.0, 2.0, 5.0, 10.0]

state = {}
input_widgets = {}


def fmt_num(x, sig=4):
    s = f"{x:.{sig}g}"
    if 'e' not in s and '.' in s:
        s = s.rstrip('0').rstrip('.')
    return s


def fmt_ohm(x):
    if abs(x) >= 1e6:
        return f"{fmt_num(x/1e6)} MOhm"
    if abs(x) >= 1e3:
        return f"{fmt_num(x/1e3)} kOhm"
    if abs(x) < 1 and abs(x) > 0:
        return f"{fmt_num(x*1e3)} mOhm"
    return f"{fmt_num(x)} Ohm"


def fmt_len_m(x):
    if x >= 1:
        return f"{fmt_num(x)} m"
    if x >= 1e-2:
        return f"{fmt_num(x*100)} cm"
    return f"{fmt_num(x*1000)} mm"


def format_temp(t):
    return f"{int(t)} ∞C"


def area_from_geom(kind, val_mm):
    if kind == 'r':
        return math.pi * val_mm * val_mm
    if kind == 'd':
        return math.pi * (val_mm / 2.0) ** 2
    return val_mm * val_mm

def describe_geometry(c):
    kind = c.get('geom_kind')
    val = c.get('geom_val_mm')
    if kind is None or val is None:
        return 'Querschnitt: A zu dimensionieren'
    if kind == 'r':
        return f'Querschnitt rund: Radius r = {fmt_num(val)} mm'
    if kind == 'd':
        return f'Querschnitt rund: Durchmesser d = {fmt_num(val)} mm'
    return f'Querschnitt quadratisch: Seitenl‰nge a = {fmt_num(val)} mm'


def stable_seed(seed_text):
    s = str(seed_text).strip()
    if s.isdigit():
        return int(s)
    return random.randint(100000, 999999)




NETLIST_TEMPLATES = {
    'V_no_load': r'''V1 n1 0; down=2, v^<={$V_0$}, i_>={$I_0$}
R1 n1 n2; right=3, l={$R_L$}, v^>={$V_{R_L}$}
W1 n2 n3; down=2
W2 n3 0; left=3
''',
    'V_with_load': r'''V1 n1 0; down=2, v^<={$V_0$}, i_>={$I_0$}
R1 n1 n2; right=3, l={$R_L$}, v^>={$V_{R_L}$}
R2 n2 n3; down=2, l={$R_V$}, v^>={$V_{R_V}$}
W1 n3 0; left=3
''',
    'I_no_load': r'''I1 n1 0; down=2, i_>={$I_0$}
R1 n1 n2; right=3, l={$R_L$}, v^>={$V_{R_L}$}
W1 n2 n3; down=2
W2 n3 0; left=3
''',
    'I_with_load': r'''I1 n1 0; down=2, i_>={$I_0$}
R1 n1 n2; right=3, l={$R_L$}, v^>={$V_{R_L}$}
R2 n2 n3; down=2, l={$R_V$}, v^>={$V_{R_V}$}
W1 n3 0; left=3
''',
}



def netlist_key(c):
    has_load = c.get('RL_ohm', 0.0) > 0
    return f"{c['source_type']}_{'with_load' if has_load else 'no_load'}"


def build_schematic_netlist(c):
    return NETLIST_TEMPLATES[netlist_key(c)]


def render_schematic(c):
    try:
        from lcapy import Circuit
    except Exception:
        return None, 'lcapy/LaTeX nicht verf¸gbar.'

    try:
        out_dir = Path('schematics/leiter')
        out_dir.mkdir(parents=True, exist_ok=True)
        key = netlist_key(c)
        out_path = out_dir / f"leiter_{c['seed']}_{key}.png"

        cct = Circuit(build_schematic_netlist(c))

        cct['R1'].value = c['result']['RT']
        if c.get('RL_ohm', 0.0) > 0:
            cct['R2'].value = c['RL_ohm']

        if c['source_type'] == 'V':
            cct['V1'].value = c['source_value']
        else:
            cct['I1'].value = c['source_value']

        cct.draw(str(out_path), style='european', label_ids=False, label_values=False, label_nodes=False, draw_nodes=False, node_spacing=1.2, cpt_size=1.1)
        return out_path, ''
    except Exception as exc:
        return None, f'Schaltplan konnte nicht erzeugt werden: {exc}'


def solve_case(c):
    rho = c['material']['rho']
    alpha = c['material']['alpha']
    L = c['L_m']
    T1 = c['T_C']
    RL = c['RL_ohm']

    if c['mode'] == 'basis':
        A = c['A_mm2']
    else:
        A = c['A_dim_mm2']

    R20 = rho * L / A
    RT = R20 * (1.0 + alpha * (T1 - 20.0))

    if c['source_type'] == 'V':
        U0 = c['source_value']
        I20 = U0 / (R20 + RL)
        IT = U0 / (RT + RL)
        U0T0 = U0
    else:
        I0 = c['source_value']
        I20 = I0
        IT = I0
        U0T0 = I20 * (R20 + RL)

    S20 = I20 / A

    V_RL0 = I20 * R20
    P_leiter_20 = I20 * I20 * R20
    P_load_20 = I20 * I20 * RL
    P0_20 = U0T0 * I20

    return {
        'A_mm2': A,
        'R20': R20,
        'RT': RT,
        'I20': I20,
        'IT': I20,
        'S20': S20,
        'U0T': U0T0,
        'V_RL0': V_RL0,
        'P_leiter': P_leiter_20,
        'P_load': P_load_20,
        'P0': P0_20,
        'leiter_typ': 'Kaltleiter' if c['material']['alpha'] > 0 else 'Warmleiter'
    }


def generate_case(seed_text=''):
    seed = stable_seed(seed_text)
    rng = random.Random(seed)

    # Topology weights: V+RL+RV 30%, I+RL+RV 30%, V+RL 20%, I+RL 20%
    def pick_topo():
        r = rng.random()
        if r < 0.30:
            return ('V', True)
        if r < 0.60:
            return ('I', True)
        if r < 0.80:
            return ('V', False)
        return ('I', False)

    for _ in range(500):
        source_type, with_load = pick_topo()
        # Dimension mode only for V + RL + RV
        if source_type == 'V' and with_load:
            mode = rng.choice(['basis', 'dimension'])
        else:
            mode = 'basis'

        material = rng.choice(MATERIALS)
        L_m = rng.choice(L_VALUES)
        T_C = rng.randrange(-25, 101, 5)

        if mode == 'basis':
            geom_kind = rng.choice(['r', 'd', 'x'])
            geom_val_mm = rng.choice(GEOM_VALUES_MM)
            A_mm2 = area_from_geom(geom_kind, geom_val_mm)

            source_value = rng.choice(V_VALUES if source_type == 'V' else I_VALUES)
            RL_ohm = rng.choice(RL_VALUES) if with_load else 0.0

            c = {
                'seed': seed,
                'mode': mode,
                'material': material,
                'L_m': L_m,
                'T_C': T_C,
                'geom_kind': geom_kind,
                'geom_val_mm': geom_val_mm,
                'A_mm2': A_mm2,
                'source_type': source_type,
                'source_value': source_value,
                'RL_ohm': RL_ohm,
            }
            r = solve_case(c)
            if not (1e-3 < r['R20'] < 1e6):
                continue
            if not (0.01 <= r['S20'] <= 50):
                continue
            c['result'] = r
            return c

        # dimension mode (V + RL in series)
        source_value = rng.choice(V_VALUES)
        RL_ohm = rng.choice(RL_VALUES)
        S_target = rng.choice(S_TARGET_VALUES)

        rhoL = material['rho'] * L_m
        # S = U0 / (rho*L + RL*A) -> A = (U0/S - rho*L)/RL
        A_dim = (source_value / S_target - rhoL) / RL_ohm
        if not (0.05 <= A_dim <= 400):
            continue

        c = {
            'seed': seed,
            'mode': mode,
            'material': material,
            'L_m': L_m,
            'T_C': T_C,
            'source_type': source_type,
            'source_value': source_value,
            'RL_ohm': RL_ohm,
            'S_target': S_target,
            'A_dim_mm2': A_dim,
        }
        r = solve_case(c)
        if not (1e-3 < r['R20'] < 1e6):
            continue
        c['result'] = r
        return c

    raise RuntimeError('Keine g¸ltige Aufgabe gefunden.')


def clear_styles():
    for w in input_widgets.values():
        w.layout.border = ''


def set_ok(w, ok):
    w.layout.border = '2px solid #2ea043' if ok else '2px solid #cf222e'


def _mk_float(key):
    w = widgets.FloatText(layout=widgets.Layout(width='170px'))
    input_widgets[key] = w
    return w


def build_inputs(c):
    input_widgets.clear()
    rows = []
    if c['mode'] == 'basis':
        rows.append(widgets.HBox([widgets.HTML('<div style="min-width:340px">1) R<sub>L</sub>(T<sub>0</sub>) [Ohm]:</div>'), _mk_float('R20')]))
        rows.append(widgets.HBox([widgets.HTML('<div style="min-width:340px">2) Stromdichte S<sub>0</sub> [A/mm≤]:</div>'), _mk_float('S20')]))
        if c['source_type'] == 'V':
            rows.append(widgets.HBox([widgets.HTML('<div style="min-width:340px">3) Strom I<sub>0</sub>(T<sub>0</sub>) [A]:</div>'), _mk_float('IT')]))
        else:
            rows.append(widgets.HBox([widgets.HTML('<div style="min-width:340px">3) Spannung V<sub>R_L</sub>(T<sub>0</sub>) [V]:</div>'), _mk_float('V_RL0')]))
    else:
        rows.append(widgets.HTML('<b>1) Formelaufgabe (symbolisch, keine Eingabe)</b>'))
        rows.append(widgets.HBox([widgets.HTML('<div style="min-width:340px">2) Dimensionierter Querschnitt A [mm≤]:</div>'), _mk_float('A_dim')]))
        rows.append(widgets.HBox([widgets.HTML('<div style="min-width:340px">3) R<sub>L</sub>(T<sub>0</sub>) [Ohm]:</div>'), _mk_float('R20')]))
        rows.append(widgets.HBox([widgets.HTML('<div style="min-width:340px">4) Stromdichte S<sub>0</sub> [A/mm≤]:</div>'), _mk_float('S20')]))
        rows.append(widgets.HBox([widgets.HTML('<div style="min-width:340px">5) Strom I<sub>0</sub>(T<sub>0</sub>) [A]:</div>'), _mk_float('IT')]))

    has_load = c.get('RL_ohm', 0.0) > 0
    if has_load:
        if c['mode'] == 'basis':
            rows.append(widgets.HBox([widgets.HTML('<div style="min-width:340px">4) Verlustleistung Leiter P<sub>L</sub>(T<sub>0</sub>) [W]:</div>'), _mk_float('P_leiter')]))
            rows.append(widgets.HBox([widgets.HTML('<div style="min-width:340px">5) Leistung Verbraucher P<sub>V</sub>(T<sub>0</sub>) [W]:</div>'), _mk_float('P_load')]))
            rows.append(widgets.HBox([widgets.HTML('<div style="min-width:340px">6) Quellenleistung P<sub>0</sub>(T<sub>0</sub>) [W]:</div>'), _mk_float('P0')]))
            rows.append(widgets.HBox([widgets.HTML('<div style="min-width:340px">7) R<sub>L</sub>(T<sub>1</sub>) [Ohm]:</div>'), _mk_float('RT')]))
            label_typ = '8)'
        else:
            rows.append(widgets.HBox([widgets.HTML('<div style="min-width:340px">6) Verlustleistung Leiter P<sub>L</sub>(T<sub>0</sub>) [W]:</div>'), _mk_float('P_leiter')]))
            rows.append(widgets.HBox([widgets.HTML('<div style="min-width:340px">7) Leistung Verbraucher P<sub>V</sub>(T<sub>0</sub>) [W]:</div>'), _mk_float('P_load')]))
            rows.append(widgets.HBox([widgets.HTML('<div style="min-width:340px">8) Quellenleistung P<sub>0</sub>(T<sub>0</sub>) [W]:</div>'), _mk_float('P0')]))
            rows.append(widgets.HBox([widgets.HTML('<div style="min-width:340px">9) R<sub>L</sub>(T<sub>1</sub>) [Ohm]:</div>'), _mk_float('RT')]))
            label_typ = '10)'
    else:
        rows.append(widgets.HBox([widgets.HTML('<div style="min-width:340px">4) Verlustleistung Leiter P<sub>L</sub>(T<sub>0</sub>) [W]:</div>'), _mk_float('P_leiter')]))
        rows.append(widgets.HBox([widgets.HTML('<div style="min-width:340px">5) Quellenleistung P<sub>0</sub>(T<sub>0</sub>) [W]:</div>'), _mk_float('P0')]))
        rows.append(widgets.HBox([widgets.HTML('<div style="min-width:340px">6) R<sub>L</sub>(T<sub>1</sub>) [Ohm]:</div>'), _mk_float('RT')]))
        label_typ = '7)'

    nature = widgets.Dropdown(options=['Warmleiter', 'Kaltleiter'], value='Warmleiter', layout=widgets.Layout(width='200px'))
    input_widgets['leiter_typ'] = nature
    rows.append(widgets.HBox([widgets.HTML(f'<div style="min-width:340px">{label_typ} Leiter ist:</div>'), nature]))

    box_inputs.children = rows

def make_tasks(c):
    tasks = []
    has_load = c.get('RL_ohm', 0.0) > 0

    if c['mode'] == 'dimension':
        tasks.append('1) Stelle die Formel f¸r den Widerstand R<sub>L</sub>(T<sub>0</sub>) (in Abh‰ngigkeit von Geometrie und Materialdaten) und f¸r den Strom I<sub>0</sub>(T<sub>0</sub>) (abh‰ngig von Quelle, R<sub>L</sub>' + (' und R<sub>V</sub>' if has_load else '') + ') auf.')
        tasks.append('2) Dimensioniere den Leiterquerschnitt A f¸r die vorgegebene Stromdichte S<sub>0</sub> (A/mm≤).')
        tasks.append('3) Berechne den Leiterwiderstand R<sub>L</sub> bei T<sub>0</sub> = 20∞C.')
        tasks.append('4) Berechne die Stromdichte S<sub>0</sub> im Leiter bei T<sub>0</sub>.')
        tasks.append('5) Berechne den Strom I<sub>0</sub> durch den Leiter bei T<sub>0</sub>.')
        idx = 6
    else:
        tasks.append('1) Berechne den Leiterwiderstand R<sub>L</sub> bei T<sub>0</sub> = 20∞C.')
        tasks.append('2) Berechne die Stromdichte S<sub>0</sub> im Leiter bei T<sub>0</sub>.')
        if c['source_type'] == 'V':
            tasks.append('3) Berechne den Strom I<sub>0</sub> durch den Leiter bei T<sub>0</sub>.')
        else:
            tasks.append('3) Berechne die Spannung V<sub>R_L</sub> ¸ber dem Leiter bei T<sub>0</sub>.')
        idx = 4

    tasks.append(f'{idx}) Berechne die Wirkleistung P<sub>L</sub> im Leiter bei T<sub>0</sub>.')
    idx += 1
    if has_load:
        tasks.append(f'{idx}) Berechne die Wirkleistung P<sub>V</sub> im Verbraucher bei T<sub>0</sub>.')
        idx += 1
    tasks.append(f'{idx}) Berechne die von der Quelle abgegebene Leistung P<sub>0</sub> bei T<sub>0</sub>.')
    idx += 1
    tasks.append(f'{idx}) Berechne den Leiterwiderstand R<sub>L</sub> bei T<sub>1</sub>.')
    idx += 1
    tasks.append(f'{idx}) Handelt es sich um einen Warm- oder Kaltleiter?')

    return tasks

def refresh_outputs(c):
    r = c['result']
    has_load = c.get('RL_ohm', 0.0) > 0

    src_text = f"Spannungsquelle V<sub>0</sub> = {fmt_num(c['source_value'])} V" if c['source_type'] == 'V'         else f"Stromquelle I<sub>0</sub> = {fmt_num(c['source_value'])} A"

    values_lines = [
        '<b>Gegeben</b><br>',
        f"- Seed: {c['seed']}<br>",
        f"- Material: {c['material']['name']}<br>",
        f"- œÅ (T<sub>0</sub>): {fmt_num(c['material']['rho'])} Ohm∑mm≤/m<br>",
        f"- Œ±: {fmt_num(c['material']['alpha'])} 1/K<br>",
        f"- Leiterl‰nge L: {fmt_len_m(c['L_m'])}<br>",
        f"- T<sub>0</sub> = 20∞C<br>",
        f"- T<sub>1</sub> = {format_temp(c['T_C'])}<br>",
        f"- {describe_geometry(c)}<br>",
        f"- Quelle: {src_text}<br>",
    ]
    if c['mode'] == 'dimension':
        values_lines.append(f"- Vorgegebene Stromdichte S<sub>0</sub>: {fmt_num(c['S_target'])} A/mm≤<br>")
    if has_load:
        values_lines.append(f"- Verbraucher R<sub>V</sub>: {fmt_ohm(c['RL_ohm'])}<br>")
    out_values.value = ''.join(values_lines)

    out_tasks.value = '<b>Aufgaben</b><br>' + '<br>'.join(make_tasks(c))

    sol_lines = ['<b>Musterlˆsung</b><br>']
    if c['mode'] == 'basis':
        sol_lines.append(f"- R<sub>L</sub>(T<sub>0</sub>) = {fmt_ohm(r['R20'])}<br>")
        sol_lines.append(f"- S<sub>0</sub> = {fmt_num(r['S20'])} A/mm≤<br>")
        if c['source_type'] == 'V':
            sol_lines.append(f"- I<sub>0</sub>(T<sub>0</sub>) = {fmt_num(r['I20'])} A<br>")
        else:
            sol_lines.append(f"- V<sub>R_L</sub>(T<sub>0</sub>) = {fmt_num(r['V_RL0'])} V<br>")
    else:
        sol_lines.append(f"- A (dimensioniert, T<sub>0</sub>) = {fmt_num(r['A_mm2'])} mm≤<br>")
        sol_lines.append(f"- R<sub>L</sub>(T<sub>0</sub>) = {fmt_ohm(r['R20'])}<br>")
        sol_lines.append(f"- S<sub>0</sub> = {fmt_num(r['S20'])} A/mm≤<br>")
        sol_lines.append(f"- I<sub>0</sub>(T<sub>0</sub>) = {fmt_num(r['I20'])} A<br>")

    sol_lines.append(f"- P<sub>L</sub>(T<sub>0</sub>) = {fmt_num(r['P_leiter'])} W<br>")
    if has_load:
        sol_lines.append(f"- P<sub>V</sub>(T<sub>0</sub>) = {fmt_num(r['P_load'])} W<br>")
    sol_lines.append(f"- P<sub>0</sub>(T<sub>0</sub>) = {fmt_num(r['P0'])} W<br>")

    sol_lines.append(f"- R<sub>L</sub>(T<sub>1</sub>) = {fmt_ohm(r['RT'])}<br>")
    sol_lines.append(f"- Typ: {r['leiter_typ']}")

    out_solution.value = ''.join(sol_lines)

def generate(seed_text=''):
    title_html.value = f'<h2>{TITLE} [... loading ...]</h2>'
    try:
        c = generate_case(seed_text)
        state.clear()
        state.update(c)
        seed.value = str(c['seed'])
        refresh_outputs(c)
        p, msg = render_schematic(c)
        if p and p.exists():
            img_schematic.value = p.read_bytes()
            out_schematic_msg.value = ''
        else:
            img_schematic.value = b''
            out_schematic_msg.value = f'<span style="color:#cf222e">{msg}</span>' if msg else ''
        build_inputs(c)
        clear_styles()
        out_check.value = ''
    finally:
        title_html.value = f'<h2>{TITLE}</h2>'


def _ok(user, ref):
    if ref == 0:
        return user == 0
    if user == 0:
        return False
    if (user > 0) != (ref > 0):
        return False
    tol = 0.01 * abs(ref)
    return abs(user - ref) <= tol


def on_check(_):
    if not state:
        return
    c = state
    r = c['result']
    all_ok = True
    has_load = c.get('RL_ohm', 0.0) > 0

    checks = []
    if c['mode'] == 'basis':
        checks += [('R20', r['R20']), ('S20', r['S20'])]
        if c['source_type'] == 'V':
            checks.append(('IT', r['IT']))
        else:
            checks.append(('V_RL0', r['V_RL0']))
    else:
        checks += [('A_dim', r['A_mm2']), ('R20', r['R20']), ('S20', r['S20']), ('IT', r['IT'])]

    checks.append(('P_leiter', r['P_leiter']))
    if has_load:
        checks.append(('P_load', r['P_load']))
    checks.append(('P0', r['P0']))
    checks.append(('RT', r['RT']))

    for key, ref in checks:
        w = input_widgets[key]
        ok = _ok(float(w.value), ref)
        set_ok(w, ok)
        all_ok = all_ok and ok

    typ_w = input_widgets['leiter_typ']
    ok_typ = (typ_w.value == r['leiter_typ'])
    set_ok(typ_w, ok_typ)
    all_ok = all_ok and ok_typ

    out_check.value = '<b style="color:#2ea043">Alle Eingaben korrekt.</b>' if all_ok else '<b style="color:#cf222e">Mindestens eine Eingabe ist falsch.</b>'


# ---- UI ----
seed = widgets.Text(description='Seed:', placeholder='leer = random')
btn_seed = widgets.Button(description='Generieren (Seed)')
btn_default = widgets.Button(description='Generieren (Default)')
btn_random = widgets.Button(description='Generieren (Random)')
btn_check = widgets.Button(description='Pr¸fen')

# widths
btn_seed.layout = widgets.Layout(width='170px')
btn_default.layout = widgets.Layout(width='170px')
btn_random.layout = widgets.Layout(width='170px')


title_html = widgets.HTML(f'<h2>{TITLE}</h2>')
out_values = widgets.HTML()
out_tasks = widgets.HTML()
img_schematic = widgets.Image(format='png', layout=widgets.Layout(width='800px'))
out_schematic_msg = widgets.HTML()
box_inputs = widgets.VBox()
out_check = widgets.HTML()
out_solution = widgets.HTML()

btn_seed.on_click(lambda _: generate(seed.value.strip()))
btn_default.on_click(lambda _: generate('246810'))
btn_random.on_click(lambda _: generate(''))
btn_check.on_click(on_check)

ui = widgets.VBox([
    title_html,
    widgets.HBox([seed, btn_seed, btn_default, btn_random]),
    out_values,
    img_schematic,
    out_schematic_msg,
    out_tasks,
    widgets.HTML('<b>Eingaben</b>'),
    box_inputs,
    btn_check,
    out_check,
    widgets.HTML('<hr>'),
    out_solution,
])

display(ui)
generate('')

